In [1]:
using HDF5

In [2]:
global const fid::HDF5.File        = h5open("/home/nabil/work/projects/hrfile_hdf5_converter/cmake-build-debug/wann.hdf5", "r") 
global const nw::Int64             = read(fid["nw"])
global const nr::Int64             = read(fid["nr"]);

global const h::Vector{ComplexF64} = read(fid["reH"]) + im * read(fid["imH"]);
global const r::Matrix{ComplexF64} = reshape(read(fid["rvecs"]), (nr, 3));



In [ ]:
# Compute e^ikR Matrix 
function calculate_eikR_mat(ks_red::Matrix{Float64}, Rs::Matrix{ComplexF64}, nk, nr)
    eikR_mat::Matrix{ComplexF64} = zeros(ComplexF64, nk, nr)
    @inbounds Threads.@threads for ik = 1 : nk
        @inbounds for iR = 1 : nr
            eikR_mat[ik, iR] = exp(2π * im * dot(@view ks_red[ik, :], @view Rs[iR, :]))
        end
    end
    eikR_mat
end

@inline function hk(k, hvec, rvec, nw, nr)
    Rsum::Matrix{ComplexF64} = zeros(ComplexF64, nw, nw)
    @inbounds Threads.@threads for iR = 1 : nr
        eikR = exp(2π * im * dot(k, @view rvec[iR, :]))
        @inbounds for α = 1 : nw 
            @inbounds for β = 1 : nw 
                Rsum[α, β] += h[iR + nw * nr * α + nr * β] * eikR
            end
        end
    end
    return hsum
end

hk (generic function with 1 method)